In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn as sk
import torch as tt
import torchvision as ttv
import statsmodels.api as sm
import plotly.express as px
import plotly.io as pio
import nltk
import string
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukemcguinness/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukemcguinness/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lukemcguinness/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/lukemcguinness/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/lukemcguinness/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [49]:
# apply same logic as in analysis

df = pd.read_csv("loan_data.csv")
df['Approval'].replace(['Approved', 'Rejected'], [1,0], inplace=True)
df['Employment_Status'].replace(['employed', 'unemployed'], [1,0], inplace=True)

data = df[(df['DTI_Ratio'] >= 0) & 
            (df['Credit_Score'] >= 550) & 
            (df['Loan_Amount'] < 120000)]

def preprocess(text):
    text = text.lower() 
    text = text.strip()
    text = re.compile(r'[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\d',' ',text) 
    return text

def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

wl = WordNetLemmatizer()
 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) 
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for tag in word_pos_tags] 
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

data['clean_text'] = data['Text'].apply(lambda x: finalpreprocess(x))

/var/folders/sk/57wc0kxd0bld0mny_6rdm1g80000gn/T/ipykernel_24656/1037507239.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Approval'].replace(['Approved', 'Rejected'], [1,0], inplace=True)
/var/folders/sk/57wc0kxd0bld0mny_6rdm1g80000gn/T/ipykernel_24656/1037507239.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [155]:
X_text = data['clean_text']
y = data['Approval']
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)

# convert to numbers and only take 500 most impportant words ** possibly increase/decrease **
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray() # array to be handles by tensor/torch
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray() # array to be handles by tensor/torch

# Numeric features
numeric_cols = ['Income', 'Credit_Score', 'Loan_Amount', 'DTI_Ratio', 'Employment_Status']
X_train_num = data.loc[X_train.index][numeric_cols]
X_test_num = data.loc[X_test.index][numeric_cols]

# 0 mean and unit variance for input into NN
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

# now the model is more accuracte with the income included
# scaler_reduced = StandardScaler()
# X_train_num_scaled_reduced = scaler_reduced.fit_transform(X_train_num.drop(['Income'], axis=1))
# X_num_test_scaled_reduced = scaler_reduced.transform(X_test_num.drop(['Income'], axis=1))

**PyTorch Model**

In [156]:
# X_train_num_scaled_reduced = scaler.fit_transform(X_train_num.drop(['Loan_Amount'], axis=1))# 
# ^^ use to test without loan_amount ^^

class CombinedTorchModel(nn.Module):
    # initialize 3 layers
    def __init__(self, text_dim, num_dim):
        super().__init__()
        self.text_layer = nn.Linear(text_dim, 64)
        self.num_layer = nn.Linear(num_dim, 32)
        self.combined_layer = nn.Linear(96, 1)

    # pass into model
    def forward(self, text_input, num_input):
        x_text = torch.sigmoid(self.text_layer(text_input))
        x_num = torch.relu(self.num_layer(num_input))
        x = torch.cat((x_text, x_num), dim=1) # concatenates the outputs from the two branches
        return torch.sigmoid(self.combined_layer(x)) # output between 0 and 1
    
X_text_train_torch = torch.tensor(X_train_tfidf, dtype=torch.float32)
X_num_train_torch = torch.tensor(X_train_num_scaled, dtype=torch.float32)
# use unsqueeze(1) to match what model expects
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1) 

X_text_test_torch = torch.tensor(X_test_tfidf, dtype=torch.float32)
X_num_test_torch = torch.tensor(X_test_num_scaled, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [164]:
model = CombinedTorchModel(X_train_tfidf.shape[1], X_train_num_scaled.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# bundle input and target tensors so we can iterate over it in each epoch
train_dataset = TensorDataset(X_text_train_torch, X_num_train_torch, y_train_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # split and shuffle

for epoch in range(10):
    model.train()
    # pass through training set each epoch
    for text_batch, num_batch, label_batch in train_loader:
        optimizer.zero_grad()
        output = model(text_batch, num_batch)
        loss = criterion(output, label_batch)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_output = model(X_text_test_torch, X_num_test_torch)
        val_loss = criterion(val_output, y_test_torch)
    print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

model.eval()
with torch.no_grad():
    y_pred_probs = model(X_text_test_torch, X_num_test_torch).numpy()
    y_pred = (y_pred_probs > 0.4).astype(int)

# Apply post-model auto-rejection rules
X_test_num_reset = X_test_num.reset_index(drop=True)
for i, row in X_test_num_reset.iterrows():
    if row['Loan_Amount'] >= 120000 or row['Credit_Score'] < 550 or row['DTI_Ratio'] > 50:
        y_pred[i] = 0

print("\nPyTorch Classification Report:")
print(classification_report(y_test, y_pred))


accuracy = accuracy_score(y_test, y_pred)
print(f"PyTorch Accuracy: {accuracy:.3f}")

Epoch 1, Train Loss: 0.0898, Val Loss: 0.2038
Epoch 2, Train Loss: 0.0535, Val Loss: 0.1201
Epoch 3, Train Loss: 0.3247, Val Loss: 0.0987
Epoch 4, Train Loss: 0.1009, Val Loss: 0.0847
Epoch 5, Train Loss: 0.0004, Val Loss: 0.0756
Epoch 6, Train Loss: 0.0039, Val Loss: 0.0687
Epoch 7, Train Loss: 0.0012, Val Loss: 0.0635
Epoch 8, Train Loss: 0.0000, Val Loss: 0.0595
Epoch 9, Train Loss: 0.0000, Val Loss: 0.0553
Epoch 10, Train Loss: 0.0000, Val Loss: 0.0507

PyTorch Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1701
           1       0.98      0.99      0.98       820

    accuracy                           0.99      2521
   macro avg       0.99      0.99      0.99      2521
weighted avg       0.99      0.99      0.99      2521

PyTorch Accuracy: 0.990


**TensorFlow Model**

In [167]:
input_text = Input(shape=(X_train_tfidf.shape[1],))
input_num = Input(shape=(X_train_num_scaled.shape[1],))
text_branch = Dense(64, activation='relu')(input_text)
num_branch = Dense(32, activation='relu')(input_num)
combined = Concatenate()([text_branch, num_branch])
output = Dense(1, activation='sigmoid')(combined)

model_tf = Model(inputs=[input_text, input_num], outputs=output)
model_tf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_tf.fit([X_train_tfidf, X_train_num_scaled], y_train, epochs=10, batch_size=32, verbose=1)

y_pred_tf = (model_tf.predict([X_test_tfidf, X_test_num_scaled]) > 0.5).astype(int)

# # Apply post-model auto-rejection rules
# for i, row in enumerate(X_test_num):
#     if row['Loan_Amount'] >= 120000 or row['Credit_Score'] < 550 or row['DTI_Ratio'] > 50:
#         y_pred_tf[i] = 0

print("\nTensorFlow Classification Report:")
print(classification_report(y_test, y_pred_tf))

accuracy_tf = accuracy_score(y_test, y_pred_tf)
print(f"TensorFlow Accuracy: {accuracy_tf:.3f}")

Epoch 1/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 578us/step - accuracy: 0.7576 - loss: 0.5056
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - accuracy: 0.9570 - loss: 0.1669
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - accuracy: 0.9656 - loss: 0.1163
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - accuracy: 0.9713 - loss: 0.0931
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - accuracy: 0.9768 - loss: 0.0790
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.9815 - loss: 0.0687
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step - accuracy: 0.9821 - loss: 0.0654
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.9834 - loss: 0.0617
Epoch 9/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.9850 - loss: 0.0584
Epoch 10/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.9884 - loss: 0.0549
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step

TensorFlow Classification Report:
              precision    recall  f1-score